In [5]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

gr.close_all()

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

MODEL = "gpt-4o-mini"
openai = OpenAI()

# Define your tools (functions)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "Returns the ticket price for a given city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the ticket price for"
                    }
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Returns the current weather for a given city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather for"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

# Dummy tool call handler
def handle_tool_call(tool_call):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    city = args.get("city", "unknown")

    if name == "get_ticket_price":
        return f"The ticket price to {city} is $799."
    elif name == "get_weather":
        return f"The current weather in {city} is sunny and 25°C."
    else:
        return "Unknown tool."

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

# Chat function with multiple tool call support
def chat(message, history):
    smessage = {"role": "system", "content": system_message}
    messages = [smessage] + history + [{"role": "user", "content": message}]
    
    # First model call
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    reply = response.choices[0].message
    print(reply.tool_calls)
    # If tool calls are requested
    if response.choices[0].finish_reason=="tool_calls":
        tool_calls = reply.tool_calls

        for tool_call in tool_calls:
            tool_response = handle_tool_call(tool_call)
            print(tool_response)
            # Append tool call
            messages.append({
                "role": "assistant",
                "tool_call_id": tool_call.id,
                "content": None,
                "tool_calls": [tool_call]
            })

            # Append tool response
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": tool_response
            })

        # Second model call with tool responses
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages
        )

    return response.choices[0].message.content

In [6]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7927
* To create a public link, set `share=True` in `launch()`.


None
None
[ChatCompletionMessageToolCall(id='call_DkKWUdJOllBJPke8zTuLgfuY', function=Function(arguments='{"city": "Tokyo"}', name='get_ticket_price'), type='function'), ChatCompletionMessageToolCall(id='call_zs0bRJ6xF1mPHEFFZf14I5dH', function=Function(arguments='{"city": "Tokyo"}', name='get_weather'), type='function')]
The ticket price to Tokyo is $799.
The current weather in Tokyo is sunny and 25°C.
